#### Initializing Dims and Facts (once only)
* Create empty table for facts. Dims will be added to faa.airport_dims
* Create dims stored procedures and execute
* Create facts stored procedures and execute  

#### Updating Dims and Facts with new data
* Run R script to prep data for staging area ([elmer_prep_faa_cargo.R](https://github.com/psrc/trends-airports/blob/master/elmer/elmer_prep_faa_cargo.R))
* Execute dims stored procedures
* Execute facts stored procedures

In [4]:
SELECT top 10 *
FROM Sandbox.Christy.faa_cargo;

In [1]:
SELECT DISTINCT [YEAR]
FROM Sandbox.Christy.faa_cargo;

(10 rows affected)

Total execution time: 00:00:00.043

YEAR
2015
2014
2009
2013
2008
2012
2011
2016
2010
2017


In [18]:
SELECT *
FROM Sandbox.Christy.faa_cargo
WHERE year = 2014;

(132 rows affected)

Total execution time: 00:00:00.031

Rank,Locid,RO,ST,City,Airportname,SL,Hub,landed_weight,year
1,MEM,SO,TN,Memphis,Memphis International,NULL,M,22774592279,2014
2,ANC,AL,AK,Anchorage,Ted Stevens Anchorage International,NULL,M,15867941046,2014
3,SDF,SO,KY,Louisville,Louisville International-Standiford Field,NULL,S,11568369154,2014
4,ORD,GL,IL,Chicago,Chicago O'Hare International,NULL,L,7541411779,2014
5,MIA,SO,FL,Miami,Miami International,NULL,L,7192790882,2014
6,IND,GL,IN,Indianapolis,Indianapolis International,NULL,M,5355984715,2014
7,LAX,WP,CA,Los Angeles,Los Angeles International,NULL,L,4297359912,2014
8,CVG,SO,KY,Greater Cincinnati International Airport,Cincinnati/Northern Kentucky International,NULL,M,3644404568,2014
9,JFK,EA,NY,New York,John F Kennedy International,NULL,L,3170996874,2014
10,DFW,SW,TX,Fort Worth,Dallas/Fort Worth International,NULL,L,3140733270,2014


In [13]:
SELECT *
FROM stg.faa_cargo;

# Dimensions

Create/alter dims stored procedure that adds on to faa.aiport_dims

In [14]:
ALTER PROCEDURE [faa].[merge_cargo_dim]
as

;with cte as (
	select ISNULL(RO, '--') AS RO, 
		ISNULL(ST, '--') AS ST, 
		ISNULL(Locid, '--') as Locid, 
		ISNULL(City, '--') as City, 
		ISNULL(Airportname, '--') as Airportname, 
		ISNULL(SL, '--') as SL, 
		ISNULL(Hub, '--') as Hub
	from Sandbox.Christy.faa_cargo
    --from stg.faa_cargo
	group by RO, ST, Locid, City, Airportname, SL, Hub
)
MERGE  faa.airport_dims as target
USING cte as source
ON (
	target.ro = source.RO
	and target.st = source.ST
	and target.loc_id = source.Locid
	and target.city_name = source.City
	and target.airport_name = source.Airportname
	and target.sl = source.SL
	and target.hub = source.Hub
)
WHEN NOT MATCHED THEN 
INSERT (ro, st, loc_id, city_name, airport_name, sl, hub)
VALUES (source.RO, 
	source.ST,
	source.Locid,
	source.City,
	source.Airportname,
	source.SL,
	source.Hub
);

Commands completed successfully.

Total execution time: 00:00:00.014

Run procedure

In [15]:
Exec faa.merge_cargo_dim;

(1 row affected)

Total execution time: 00:00:00.068

Check faa.airport_dim

In [2]:
SELECT *
FROM Elmer.faa.airport_dims
WHERE loc_id = 'MEM';

(3 rows affected)

Total execution time: 00:00:00.120

airport_dim_id,ro,st,loc_id,city_name,airport_name,sl,hub
816,SO,TN,MEM,Memphis,Memphis International,P,M
817,SO,TN,MEM,Memphis,Memphis International,P,S
1135,SO,TN,MEM,Memphis,Memphis International,--,M


In [5]:
SELECT *
FROM Elmer.faa.cargo_facts
WHERE data_year = '2015';

(134 rows affected)

Total execution time: 00:00:00.024

cargo_fact_id,airport_dim_id,data_year,landed_weight
7,66,2015,17139250601
16,108,2015,133685593
25,238,2015,282204502
34,240,2015,461908212
38,257,2015,211148324
52,273,2015,503007188
61,275,2015,196625120
70,276,2015,326316652
79,297,2015,365487434
87,301,2015,520180931


# Facts

Create Facts table

In [1]:
CREATE TABLE [faa].[cargo_facts](
	[cargo_fact_id] [int] IDENTITY(1,1) NOT NULL,
	[airport_dim_id] [int] NOT NULL,
	[data_year] [smallint] NOT NULL,
	[landed_weight] [float] NOT NULL,
 CONSTRAINT [pk_cargo_facts] PRIMARY KEY NONCLUSTERED 
(
	[airport_dim_id] ASC,
	[data_year] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
) ON [PRIMARY]
GO
CREATE CLUSTERED INDEX [idx_cargo_facts_clustered] ON [faa].[cargo_facts]
(
	[cargo_fact_id] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'cargo_facts' in the database.

Commands completed successfully.

Total execution time: 00:00:00.027

Check empty table

In [2]:
SELECT * 
FROM Elmer.faa.cargo_facts;

(0 rows affected)

Total execution time: 00:00:00.149

cargo_fact_id,airport_dim_id,data_year,landed_weight


#### Alter a stored procedure for merging FAA Cargo Facts

In [21]:
ALTER PROCEDURE [faa].[merge_cargo_facts] 
as

;with cte as (
	select ad.airport_dim_id, fe.landed_weight, fe.year
    from stg.faa_cargo as fe 
	--from Sandbox.Christy.faa_cargo as fe
        INNER JOIN Elmer.faa.airport_dims as ad
        ON fe.RO = ad.ro
        and fe.ST = ad.st
        and fe.Locid = ad.loc_id
        and fe.City = ad.city_name
        and fe.Airportname = ad.airport_name
        and ISNULL(fe.SL, '--') = ad.sl
        and ISNULL(fe.Hub, '--') = ad.hub
)
MERGE faa.cargo_facts as target
USING cte as source
ON (target.airport_dim_id = source.airport_dim_id
	and target.landed_weight = source.landed_weight
	and target.data_year = source.year
)
WHEN NOT MATCHED THEN 
INSERT (airport_dim_id, data_year, landed_weight)
VALUES (source.airport_dim_id,
    source.year, 
	source.landed_weight
);

Commands completed successfully.

Total execution time: 00:00:00.013

Run stored procedure

In [22]:
EXEC faa.merge_cargo_facts;

(132 rows affected)

Total execution time: 00:00:00.051

In [6]:
SELECT ad.*, cf.*
FROM faa.cargo_facts as cf 
JOIN faa.airport_dims as ad
ON cf.airport_dim_id = ad.airport_dim_id
WHERE data_year = '2014';
--WHERE city_name = 'Memphis';

(0 rows affected)

Total execution time: 00:00:00.020

airport_dim_id,ro,st,loc_id,city_name,airport_name,sl,hub,cargo_fact_id,airport_dim_id,data_year,landed_weight


In [8]:
SELECT *
FROM faa.cargo_facts
WHERE data_year = '2014';

(0 rows affected)

Total execution time: 00:00:00.015

cargo_fact_id,airport_dim_id,data_year,landed_weight


In [20]:
USE Elmer

Commands completed successfully.

Total execution time: 00:00:00.010